# Домашнее задание 2 по обработке текстов

Рассмотрим задачу бинарной классификации. Пусть дано два списка имен: мужские и женские имена. Требуется разработать классификатор, который по данному имени будет определять мужское оно или женское.

Данные: 
* Женские имена: female.txt
* Мужские имена: male.txt

## Часть 1. Предварительная обработка данных

1. Удалите неоднозначные имена (те имена, которые являются и мужскими, и женскими дновременно), если такие есть; 
2. Создайте обучающее и тестовое множество так, чтобы в обучающем множестве классы были сбалансированы, т.е. к классу принадлежало бы одинаковое количество имен;

In [1]:
import pandas as pd

In [2]:
female = open('female.txt', 'r')

In [3]:
male = open('male.txt', 'r')

In [4]:
female = [name.strip() for name in female]

In [5]:
male = [name.strip() for name in male]

In [6]:
for name in female:
    if name in male:
        female.remove(name)

In [7]:
for name in male:
    if name in female:
        male.remove(name)

In [8]:
len(male)

2892

In [9]:
len(female)

4686

In [10]:
female = female[:2893]

In [11]:
from sklearn.model_selection import train_test_split


In [12]:
male_train, male_test= train_test_split(male, test_size=0.25, random_state=10)

In [13]:
female_train, female_test= train_test_split(female, test_size=0.25, random_state=10)

In [14]:
male_df_train = pd.DataFrame(male_train, columns=['name'])

In [15]:
male_df_train['is_male'] = 1

In [16]:
female_df_train = pd.DataFrame(female_train, columns=['name'])

In [17]:
female_df_train['is_male'] = 0

In [18]:
male_df_test = pd.DataFrame(male_test, columns=['name'])

In [19]:
male_df_test['is_male'] = 1

In [20]:
female_df_test = pd.DataFrame(female_test, columns=['name'])

In [21]:
female_df_test['is_male'] = 0

In [22]:
train_frames = [male_df_train, female_df_train]

In [23]:
train_data =  pd.concat(train_frames, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)

In [24]:
test_frames = [male_df_test, female_df_test]

In [25]:
test_data =  pd.concat(test_frames, axis=0, join='outer', join_axes=None, ignore_index=False,
          keys=None, levels=None, names=None, verify_integrity=False,
          copy=True)

##  Часть 2. Базовый метод классификации

Используйте метод наивного Байеса или логистическую регрессию для классификации имен: в качестве признаков используйте символьные $n$-граммы. Сравните результаты, получаемые при разных $n=2,3,4$ по $F$-мере и аккуратности. В каких случаях метод ошибается?

Для генерации $n$-грамм используйте:

In [26]:
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1,4),max_features=1456)
from sklearn.linear_model import LogisticRegression

In [27]:
X_train = count_vect.fit_transform(train_data.name)

In [28]:
X_train

<4338x1456 sparse matrix of type '<class 'numpy.int64'>'
	with 1483 stored elements in Compressed Sparse Row format>

In [29]:
X_test = count_vect.fit_transform(test_data.name)

In [30]:
X_test

<1447x1456 sparse matrix of type '<class 'numpy.int64'>'
	with 1470 stored elements in Compressed Sparse Row format>

In [31]:
clf = LogisticRegression().fit(X_train, train_data.is_male)

In [32]:
y_pred = clf.predict(X_test)

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
accuracy_score(test_data.is_male, y_pred)

0.63234277816171391

In [35]:
from sklearn.naive_bayes import MultinomialNB

In [36]:
clf = MultinomialNB().fit(X_train, train_data.is_male)

In [37]:
y_pred = clf.predict(X_test)

In [38]:
accuracy_score(test_data.is_male, y_pred)

0.63372494816862479

##  Часть 3. Нейронная сеть


Используйте  реккурентную нейронную сеть с  LSTM для решения задачи. В ней может быть несколько слоев с LSTM, несколько слоев c Bidirectional(LSTM).  У нейронной сети один выход, определяющий класс имени. 

Представление имени для классификации в этом случае: бинарная матрица размера (количество букв в алфавите $\times$ максимальная длина имени). Обозначим его через $x$. Если первая буква имени a, то $x[1][1] = 1$, если вторая – b, то  $x[2][1] = 1$.  

Не забудьте про регуляризацию нейронной сети дропаутами. 

Сравните результаты классификации разными методами. Какой метод лучше и почему?

Сравните результаты, получаемые при разных значениях дропаута, разных числах узлов на слоях нейронной сети по $F$-мере и аккуратности. В каких случаях нейронная сеть ошибается?

In [39]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras import __version__ as keras_version
import numpy as np
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.
/Users/alex/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [40]:
nEpochs = 10
weightsFileName = "gender_weights.h5"
with open("male.txt") as f:
    m_names = f.readlines()

with open("female.txt") as f:
    f_names = f.readlines()

mf_names = []

for f_name in f_names:
    if f_name in m_names:
        mf_names.append(f_name)

m_names = [m_name.lower() for m_name in m_names if not m_name in mf_names]
f_names = [f_name.lower() for f_name in f_names if not f_name in mf_names]


totalEntries = len(m_names) + len(f_names)
maxlen = len(max( m_names , key=len)) + len(max( f_names , key=len))

chars = set(  "".join(m_names) + "".join(f_names)  )
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


print ("total endtries ") , totalEntries
print ("max len ") , maxlen
print(('total chars:'), len(chars))

total endtries 
max len 
total chars: 30


In [41]:
X = np.zeros((totalEntries , maxlen, len(chars) ), dtype=np.bool)
y = np.zeros((totalEntries , 2 ), dtype=np.bool)


for i, name in enumerate(m_names):
    for t, char in enumerate(name):
        X[i, t, char_indices[char]] = 1
    y[i, 0 ] = 1

for i, name in enumerate(f_names):
    for t, char in enumerate(name):
        X[i + len(m_names), t, char_indices[char]] = 1
    y[i + len(m_names) , 1 ] = 1

def vec2c(vec):
	for i,v in enumerate(vec):
		if v:
			return indices_char[i]
	return ""

print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(chars))))
model.add(Dropout(0.5))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.4))


model.add(BatchNormalization())
model.add(Dense(128))
model.add(Dropout(0.2))
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam')


json_string = model.to_json()

with open("model.json", "w") as text_file:
    text_file.write(json_string)


if keras_version[0] == '1':
	model.fit(X, y, batch_size=16, nb_epoch=nEpochs, validation_split=0.2)
else:
	model.fit(X, y, batch_size=16, epochs=nEpochs,validation_split=0.2)

model.save_weights('my_model_weights.h5')

print ("done and weights saved")
score = model.evaluate(X, y, batch_size=16)
print ("score ") , score

Build model...
Train on 5771 samples, validate on 1443 samples
Epoch 1/10
5771/5771 [==============================] - 155s 27ms/step - loss: 0.6064 - val_loss: 0.2666
Epoch 2/10
5771/5771 [==============================] - 145s 25ms/step - loss: 0.4917 - val_loss: 1.8980
Epoch 3/10
5771/5771 [==============================] - 162s 28ms/step - loss: 0.4427 - val_loss: 2.3584
Epoch 4/10
5771/5771 [==============================] - 148s 26ms/step - loss: 0.3822 - val_loss: 3.5314
Epoch 5/10
5771/5771 [==============================] - 143s 25ms/step - loss: 0.3538 - val_loss: 3.3669
Epoch 6/10
5771/5771 [==============================] - 143s 25ms/step - loss: 0.3404 - val_loss: 2.5924
Epoch 7/10
5771/5771 [==============================] - 144s 25ms/step - loss: 0.3282 - val_loss: 2.2495
Epoch 8/10
5771/5771 [==============================] - 144s 25ms/step - loss: 0.3260 - val_loss: 3.5103
Epoch 9/10
5771/5771 [==============================] - 144s 25ms/step - loss: 0.2983 - val_loss:

(None, 0.88974247840399689)

Преобучился?